In [1]:
%load_ext autoreload
%autoreload 2

%cd '..'

/dss/dsshome1/03/di93fup/polarization_reddit


In [2]:
import logging

# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     handlers=[
#         logging.FileHandler("data/logs/mass_shootings_preprocess_comments_notebook.log"),
#         logging.StreamHandler()
#     ]
# )

In [3]:
from tqdm import tqdm
tqdm.pandas()

from load.utils import (
    load_df_from_parquet,
)
from preprocessing.utils import (
    tokenize_comment,
    save_event_comments,
    build_vocab,
    save_event_vocab,
)

from preprocessing.constants import (
    MASS_SHOOTINGS_EVENTS_INFO,
    MASS_SHOOTINGS_KEYWORDS,
    MIN_OCCURENCE_FOR_VOCAB,
    get_event_regex,
)

In [4]:
YEARS = [2015, 2016, 2017, 2018]
START_MONTH = 1
STOP_MONTH = 12

EVENT_NAMES = list(MASS_SHOOTINGS_EVENTS_INFO.keys())

MASS_SHOOTING_EVENTS_KEYWORDS = [
    keyword
    for event_name in EVENT_NAMES
    for keyword in MASS_SHOOTINGS_EVENTS_INFO[event_name]["keywords"]
]


In [5]:
MASS_SHOOTING_REGEX = get_event_regex(
    MASS_SHOOTINGS_KEYWORDS,
    MASS_SHOOTING_EVENTS_KEYWORDS,
    operator="and",
)

logging.info(MASS_SHOOTING_REGEX)


## Filter event comments

In [6]:
logging.info("Load partisan comments")
if len(YEARS) == 1:
    partisan_comments = load_df_from_parquet(
        file_name=f"user_partisan_comments_{YEARS[0]}.parquet",
    )
else:
    partisan_comments = load_df_from_parquet(
        file_name=f"partisan_comments_{YEARS[0]}-{YEARS[-1]}.parquet",
    )


In [10]:
logging.info("Filtering event data based on keywords...")
event_comments = partisan_comments[
    partisan_comments["body_cleaned"].str.contains(
        MASS_SHOOTING_REGEX,
    )
].copy()
logging.info("finished keyword filtering")

In [12]:
event_comments.shape

(0, 7)

In [11]:
logging.info(event_comments.shape)

In [ ]:
del partisan_comments

In [ ]:
logging.info(event_comments.dtypes)

## Tokenize and stem comments

In [ ]:
# event_comments = load_event_comments("temp_mass_shootings")

In [ ]:
logging.info("Tokenizing comments...")
event_comments["tokens"] = event_comments["body_cleaned"].progress_apply(
    tokenize_comment,
).astype("string")
logging.info("Finish tokenizing comments")

In [ ]:
logging.info(event_comments.dtypes)

In [ ]:
logging.info(f"Nr of event comments: {len(event_comments)}")

In [ ]:
logging.info("saving event comments...")
save_event_comments(event_comments, "mass_shootings")

## Build event vocabulary

In [ ]:
# Read event data
# events_comments = load_event_comments(
#     EVENT_NAME,
# )

event_vocab = build_vocab(
    event_comments["tokens"],
    min_comment_freq=MIN_OCCURENCE_FOR_VOCAB,
)


In [ ]:
logging.info("Vocabulary length")
logging.info(len(event_vocab))

In [ ]:
logging.info("Saving event vocab")
save_event_vocab(event_vocab, "mass_shootings")